In [1]:
import pandas as pd
import sqlite3

## • create a connection to the database using the library sqlite3

In [2]:
con = sqlite3.connect('../data/checking-logs.sqlite.sqlite')

## • create a new table datamart in the database by joining the tables pageviews and checker using only one query
◦ the table should have the following columns: uid, labname, first_commit_ts, first_view_ts
◦ first_commit_ts is just a new name of the column timestamp from the checker table, it shows the first commit from a particular lab and from a particular user
◦ first_view_ts is the first visit of a user to the table pageviews, timestamp when a user visited the newsfeed
◦ status = ’ready’ should still be a filter
◦ numTrials = 1 should still be a filter
◦ labnames should still be from the list: ’laba04’, ’laba04s’, ’laba05’, ’laba06’, ’laba06s’, ’project1’
◦ the table should contain only the users (uids with user_*) and not the admins
◦ first_commit_ts and first_view_ts should be parsed as datetime64[ns]

In [3]:
query = """
        SELECT checker.uid, labname, timestamp AS first_commit_ts,
        datetime AS 'first_view_ts'
        FROM checker LEFT JOIN pageviews ON pageviews.uid = checker.uid
        WHERE status='ready'
               AND numTrials=1
               AND labname IN ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s', 'project1')
               AND checker.uid LIKE 'user_%'
               AND (datetime is NULL OR
                    datetime IN (SELECT MIN(datetime)
                                 FROM pageviews
                                 GROUP BY uid
                                 )
        )
        """
datamart = pd.io.sql.read_sql(query, con, parse_dates=['first_commit_ts', 'first_view_ts'])
datamart

,uid,labname,first_commit_ts,first_view_ts
0,user_4,project1,2020-04-17 05:19:02.744528,NaT
1,user_4,laba04,2020-04-17 11:33:17.366400,NaT
2,user_4,laba04s,2020-04-17 11:48:41.992466,NaT
3,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
4,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
...,...,...,...,...
135,user_23,laba06,2020-05-21 08:34:10.517205,NaT
136,user_19,laba06s,2020-05-21 13:27:06.705881,2020-04-21 20:30:38.034966
137,user_23,laba06s,2020-05-21 14:29:15.709568,NaT
138,user_17,laba06,2020-05-21 15:21:31.567615,2020-04-18 10:56:55.833899


## • using Pandas methods, create two dataframes: test and control

◦ test should have the users that have the values in first_view_ts

In [4]:
test = datamart.loc[datamart['first_view_ts'].notnull()].copy()
test

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59 entries, 3 to 139
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              59 non-null     object        
 1   labname          59 non-null     object        
 2   first_commit_ts  59 non-null     datetime64[ns]
 3   first_view_ts    59 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 2.3+ KB


◦ control should have the users that have missing values in first_view_ts

In [5]:
control = datamart.loc[datamart['first_view_ts'].isnull()].copy()
control

,uid,labname,first_commit_ts,first_view_ts
0,user_4,project1,2020-04-17 05:19:02.744528,NaT
1,user_4,laba04,2020-04-17 11:33:17.366400,NaT
2,user_4,laba04s,2020-04-17 11:48:41.992466,NaT
5,user_2,laba04,2020-04-18 13:42:35.482008,NaT
6,user_2,laba04s,2020-04-18 13:51:22.291271,NaT
...,...,...,...,...
126,user_2,laba06s,2020-05-19 14:45:03.908268,NaT
132,user_6,laba06s,2020-05-20 14:50:07.609937,NaT
134,user_7,laba06s,2020-05-20 23:05:37.742597,NaT
135,user_23,laba06,2020-05-21 08:34:10.517205,NaT


◦ replace the missing values in the control with the average first_view_ts of the test users, we will use this value for the future analysis

In [6]:
control['first_view_ts'] = test['first_view_ts'].mean()
control

,uid,labname,first_commit_ts,first_view_ts
0,user_4,project1,2020-04-17 05:19:02.744528,2020-04-27 00:40:05.761783552
1,user_4,laba04,2020-04-17 11:33:17.366400,2020-04-27 00:40:05.761783552
2,user_4,laba04s,2020-04-17 11:48:41.992466,2020-04-27 00:40:05.761783552
5,user_2,laba04,2020-04-18 13:42:35.482008,2020-04-27 00:40:05.761783552
6,user_2,laba04s,2020-04-18 13:51:22.291271,2020-04-27 00:40:05.761783552
...,...,...,...,...
126,user_2,laba06s,2020-05-19 14:45:03.908268,2020-04-27 00:40:05.761783552
132,user_6,laba06s,2020-05-20 14:50:07.609937,2020-04-27 00:40:05.761783552
134,user_7,laba06s,2020-05-20 23:05:37.742597,2020-04-27 00:40:05.761783552
135,user_23,laba06,2020-05-21 08:34:10.517205,2020-04-27 00:40:05.761783552


◦ save both tables into the database, you will use them in the next exercises

In [7]:
test.to_sql(name='test', con=con, if_exists='replace')
control.to_sql(name='control', con=con, if_exists='replace')

81

## • close the connection

In [8]:
con.close()